In [3]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [36]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "JuLdHdehKAP45u9YgmYIPoi76"
    consumer_secret = "7KmZJuoPskg4fF1n4DPloXjxfQD1R4rYx4AwGLURBP1uHCTCjz"
    access_token = "1516912179559026689-HIrvBH28htQcPqKoudWIBOWIR7W5CM"
    access_token_secret = "4de7TPCTfcf6JsCKGvw5G9oVWBmYUtbyed3crKyWvu83I"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    words = ['drinks', "drink", "shot", "beer", "alcohol", "club", "vodka", "turnup", "turnt", "lit", "drunk"]
    stream.filter(languages = ["en"], track = words)
    

{'id': 1518098155862171649, 'username': 'buffbeerleague', 'created_at': 'Sun Apr 24 05:22:59 +0000 2022', 'lng': -78.8206024, 'lat': 42.9957657, 'text': "Drinking a Starchild Mild by @ThinManBrewery @ Master Chong's World Class Tae Kwon Do — https://t.co/uhmXI5Lc5I"}
{'id': 1518098179035598848, 'username': 'connorkunz', 'created_at': 'Sun Apr 24 05:23:05 +0000 2022', 'lng': -111.54702900000001, 'lat': 39.499762000000004, 'text': 'My family was hellbent on making me watch World War Z, which I can now call one of the most incompetently shot movies ever'}
{'id': 1518098180788916226, 'username': 'Ijann__', 'created_at': 'Sun Apr 24 05:23:05 +0000 2022', 'lng': -73.1991185, 'lat': 41.1856745, 'text': 'The old me wouldve had the book lit tn😭😭 i swear .'}
{'id': 1518098412851191811, 'username': 'MattyE666', 'created_at': 'Sun Apr 24 05:24:01 +0000 2022', 'lng': 151.2374725, 'lat': -33.8847504, 'text': 'Drinking an English Bitter by @BadlandsBrewery @ The Lord Dudley Hotel — https://t.co/kl2ho

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.


In [30]:
#Example Tweets
csv_url = "https://raw.githubusercontent.com/jakobzhao/geog458/master/labs/lab02/assets/geotweets.csv"
import pandas as pd
df = pd.read_csv(csv_url)
df.to_csv(r'G:\@UW2022SPRING\458\geotweets.csv', index = False, header = True)
